# **Sentiment Analysis of Literary Texts: Code and Explanation**  
이 파일은 감정 분석에 사용한 코드의 원본을 포함하고, 이에 대한 설명, 그리고 샘플 소설에 대한 적용 예시가 포함되어 있습니다.  

**다만 python 구문을 실행하는 행위는 오류를 일으키므로, 실행하지 않고 읽기만 하시는 것을 요청합니다.**





---



## **I. 전체 코드**

우선, 전체 코드에 대해 보여드리겠습니다. 이후, 코드에 대한 설명을 하도록 하겠습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'  # 파일 경로를 필요에 맞게 수정한다.

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문단 단위로 분할 (빈 줄을 기준으로)
paragraphs = text.split('\n\n')   # 문단 단위의 분할 기준도 소설 원문의 형식에 맞게 수정한다.
paragraphs = [paragraph for paragraph in paragraphs if paragraph]  # 빈 문단 제거

# 총 문단의 개수 계산
total_paragraphs = len(paragraphs)

# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 최대 입력 길이 설정
MAX_LENGTH = 512

# 감정 분석 수행 함수 정의
def get_sentiment_score(text, prev_score=None):
    if len(text) > MAX_LENGTH:
        text = text[:MAX_LENGTH]  # 입력 문단을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(text)

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    )

    return score

# 긍정적 및 부정적 단어 리스트 정의
positive_words = [
    "happy", "joyful", "excellent", "good", "wonderful", "amazing", "great", "positive", "fortunate", "pleased",
    "delightful", "content", "cheerful", "satisfied", "thrilled", "ecstatic", "merry", "elated", "jubilant", "gleeful"
]

negative_words = [
    "sad", "terrible", "bad", "awful", "horrible", "negative", "unfortunate", "displeased", "angry", "fearful",
    "miserable", "depressed", "unhappy", "downcast", "gloomy", "dismal", "wretched", "heartbroken", "melancholy", "despair"
]

# 중요 문단을 인식하기 위한 키워드 리스트
important_keywords = [
    "conflict", "resolution", "climax", "turning point", "significant", "important", "crucial", "pivotal"
]

# 감정 분석 수행 함수 정의 (중요 문단 인식)
def get_sentiment_score(paragraph, prev_score=None):
    if len(paragraph) > MAX_LENGTH:
        paragraph = paragraph[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(paragraph)

    # 문맥에 따라 긍정 또는 부정 가중치 적용
    context_weight = 1.0
    if any(word in paragraph for word in positive_words):
        context_weight = 1.2  # 긍정 문맥 가중치
    elif any(word in paragraph for word in negative_words):
        context_weight = 0.8  # 부정 문맥 가중치

    # 중요 문단 가중치 적용
    important_weight = 1.0
    if any(keyword in paragraph for keyword in important_keywords):
        important_weight = 1.5

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합 및 가중치 적용
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    ) * context_weight * important_weight

    return score

# 첫 번째 문단은 이전 문단이 없으므로 먼저 처리
sentiment_analysis = [get_sentiment_score(paragraphs[0])]

# 나머지 문단 처리
for i in range(1, total_paragraphs):
    prev_score = sentiment_analysis[-1]
    sentiment_analysis.append(get_sentiment_score(paragraphs[i], prev_score))

# specific_size와 standard_num 계산
def calculate_specific_size(total_paragraphs, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None

    for specific_size in range(1, total_paragraphs + 1):
        standard_num = total_paragraphs // specific_size
        difference = abs(standard_num - target_standard_num)

        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num

        if closest_difference == 0:
            break

    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_paragraphs)

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_paragraphs) + 1)

# 가중 누적 평균 계산 (EWMA)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# 이동 평균의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_paragraphs)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)

# X 좌표 설정
x_sentiment = np.arange(len(sentiment_avg)) * specific_size
x_cumulative = np.arange(len(cumulative_avg_avg)) * specific_size
x_ewma = np.arange(len(ewma_avg)) * specific_size

# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.2 * std_dev_difference  # 이동 표준 편차의 1.2배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 오차 범위를 고려한 실제 문단 인덱스 계산
actual_diff_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_paragraphs, idx * specific_size + specific_size // 2)
    actual_diff_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_diff_indices = []
for start_idx, end_idx in actual_diff_indices:
    if not merged_diff_indices:
        merged_diff_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_diff_indices[-1]
        if start_idx <= last_end:
            merged_diff_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_diff_indices.append([start_idx, end_idx])

# Sentiment Volatility 계산
volatility = np.diff(sentiment_avg_smooth)

# 가우시안 스무딩 적용 (곡률을 더 크게)
volatility_smooth = gaussian_filter1d(volatility, sigma=2)

# Sentiment Volatility에서 차이가 크게 나타나는 구간 식별
volatility_diff_indices = []
for i in range(1, len(volatility_smooth) - 1):
    if (volatility_smooth[i] > volatility_smooth[i-1] and volatility_smooth[i] > volatility_smooth[i+1]) or \
       (volatility_smooth[i] < volatility_smooth[i-1] and volatility_smooth[i] < volatility_smooth[i+1]):
        volatility_diff_indices.append(i)

# 오차 범위를 고려한 실제 문단 인덱스 계산
actual_volatility_indices = []
for idx in volatility_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_paragraphs, idx * specific_size + specific_size // 2)
    actual_volatility_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_volatility_indices = []
for start_idx, end_idx in actual_volatility_indices:
    if not merged_volatility_indices:
        merged_volatility_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_volatility_indices[-1]
        if start_idx <= last_end:
            merged_volatility_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_volatility_indices.append([start_idx, end_idx])

# 그래프 그리기 (Sentiment Analysis)
plt.figure(figsize=(12, 6))

# 단순 누적 평균
plt.plot(x_cumulative, cumulative_avg_smooth, color='#DB4455', linestyle='--', label='Cumulative Average')

# 가중 누적 평균 (EWMA)
plt.plot(x_ewma, ewma_avg_smooth, color='#0E5CAD', linestyle='-', label='Exponential Weighted Moving Average')

# 이동 평균
plt.plot(x_sentiment, sentiment_avg_smooth, color='#623AA2', linestyle='-', label='Moving Average')

# S.D.P. 영역 추가
for start_idx, end_idx in merged_diff_indices:
    end_idx = min(end_idx, total_paragraphs)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

plt.xlabel('Paragraph Number')
plt.ylabel('Sentiment Polarity (-1 to 1)')
plt.title('Sentiment Analysis using Different Calculating Methods')
plt.axhline(0, color='grey', linestyle='--')  # 중립 감정선
plt.legend()

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.text(0.95, 0.10, f"max = {moving_avg_max:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')
plt.text(0.95, 0.05, f"min = {moving_avg_min:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 그래프 그리기 (Sentiment Volatility)
plt.figure(figsize=(12, 6))

# x축을 문장의 수에 맞게 조정
plt.plot(np.linspace(0, total_paragraphs, len(volatility_smooth)), volatility_smooth, color='#B4C99D', linestyle='-')
plt.axhline(0, color='grey', linestyle='--')  # y=0의 값에 회색 점선 추가

plt.xlabel('Paragraph Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')

# S.V.P. 영역 추가
for start_idx, end_idx in merged_volatility_indices:
    end_idx = min(end_idx, total_paragraphs)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {volatility_smooth.sum() * 1000:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 실제 문장 인덱스 기반 텍스트 분석
fig, ax = plt.subplots(figsize=(12, 2))
ax.text(0.5, 0.5, "<Significant Fluctuation Point>\n\nSignificant Different Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_diff_indices]) + "\n\nSignificant Volatility Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_volatility_indices]), horizontalalignment='center', verticalalignment='center', fontsize=12)
ax.axis('off')
plt.show()

# 문단별 긍정 및 부정 점수 분리
positive_scores = [score for score in sentiment_analysis if score > 0]
negative_scores = [score for score in sentiment_analysis if score < 0]

# 긍정 및 부정 점수 합산 (절댓값 사용)
positive_score_sum = sum(positive_scores)
negative_score_sum = abs(sum(negative_scores))

# 가중치 적용 (문단 수)
positive_weighted_sum = positive_score_sum * len(positive_scores)
negative_weighted_sum = negative_score_sum * len(negative_scores)

# 총합 계산
total_weighted_sum = positive_weighted_sum + negative_weighted_sum

# 백분율로 환산
positive_final_score = (positive_weighted_sum / total_weighted_sum) * 100
negative_final_score = (negative_weighted_sum / total_weighted_sum) * 100

# 막대 바 차트 시각화 코드
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_final_score, negative_final_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.2f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_final_score, negative_final_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()



---



## **II. 코드에 대한 설명**

### **1. 라이브러리 임포트 및 파일 경로 설정**  



- 분석에 필요한 라이브러리를 임포트하고, 분석할 텍스트 파일의 경로를 설정합니다.  
- 이 과정에서는 matplotlib, numpy, pandas, transformers 등 다양한 라이브러리를 사용합니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from transformers import pipeline

# 파일 경로 설정
file_path = '/주소/소설.txt'  # 임의로 바꿔서 사용하면 된다.



---



### **2. 텍스트 파일 읽기 및 문단 단위로의 분할**  



- **텍스트 파일 읽기**: 파일 경로를 설정하고 텍스트 파일을 읽어옵니다.
- **문장 분할**: 정규표현식을 사용하여 텍스트를 문단 단위로 분할합니다. 문장 단위가 아닌 문단 단위로 분할하는 이유는, 문맥을 더 잘 반영하기 위함입니다. 이때, 소설의 원문마다 문단의 분할 방식이 다를 수 있으므로 이에 유의하여 소설마다 다르게 분할하며, 빈 문단은 제거합니다.
- **총 문장 수 계산**: 전체 문단의 개수를 계산합니다.

In [ ]:
# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문단 단위로 분할 (빈 줄을 기준으로)
paragraphs = text.split('\n\n')   # 문단 단위의 분할 기준도 소설 원문의 형식에 맞게 수정한다.
paragraphs = [paragraph for paragraph in paragraphs if paragraph]  # 빈 문단 제거

# 총 문단의 개수 계산
total_paragraphs = len(paragraphs)



---



### **3. 감정 분석 모델 로드 및 감정 분석 수행**  



• 감정 분석을 실행하는 여러 모델(textblob, RoBERTa, T5 등)을 사용해 보았으나, 소설 텍스트의 감정을 가장 잘 반영한 모델이 Hugging Face의 'BERT 기반의 모델'이어서 이를 채택하였습니다.  


#### **(1) 감정 분석 모델의 제작 과정**

- 감정 분석을 최대한 정확하게 실행할 수 있는 모델을 만들기 위해서 꽤나 많은 시행착오와 과정을 거쳤습니다.  

- 자세한 사항은 **Evolution of Sentiment Analysis Models for Novels.ipynb** 파일을 확인하시길 바랍니다.



#### **(2) 감정 분석 모델의 분석 알고리즘**  

- 감정 분석을 수행할 때에는, 한 문단에 대해 감정의 점수를 매기는 방식으로 진행됩니다.
- 아래에 자세한 알고리즘을 남겨놓겠습니다. 알아보고 싶다면 읽어보시길 바랍니다. <br><br>

**문단 길이 조정:**
- 주어진 문단이 최대 길이(512자)를 초과하면 잘라서 처리합니다.<br><br>

**감정 분석 수행:**
- 감정 분석 파이프라인을 통해 문단에 대한 감정 결과를 얻습니다.<br><br>

**초기 감정 점수 설정:**
- 각 감정(anger, disgust, fear, joy, sadness)에 대한 초기 점수를 0으로 설정합니다.<br><br>

**감정 점수 할당:**
- 감정 결과를 바탕으로 각 감정의 점수를 할당합니다.
 - '1 star'는 sadness에 추가
 - '2 stars'는 fear에 추가
 - '3 stars'는 disgust에 추가
 - '4 stars'는 joy에 추가
 - '5 stars'는 joy에 두 배로 추가<br><br>

**문맥 가중치 적용:**
- 긍정적인 단어가 포함된 경우 가중치 1.2 적용
- 부정적인 단어가 포함된 경우 가중치 0.8 적용<br><br>

**중요 문단 가중치 적용:**
- 중요한 키워드가 포함된 문단은 가중치 1.5 적용<br><br>

**최종 점수 계산:**
- 감정 점수를 통합하여 최종 점수를 계산합니다.
- 최종 점수는 각 감정의 가중 합으로 계산됩니다.<br><br>

**문단 순차 처리:**
- 첫 번째 문단은 이전 점수가 없으므로 단독으로 처리합니다.  

- 나머지 문단은 이전 문단의 감정 점수를 참고하여 순차적으로 처리합니다. 이전 문단의 감정 점수가 현재 문단의 분석에 영향을 미칩니다. (문맥을 고려)  

- 이러한 과정을 통해 각 문단의 감정 점수를 계산하고, 이를 바탕으로 텍스트 전체의 감정 흐름을 분석할 수 있습니다.<br><br>


#### **(3) 감정 분석 모델 최종 코드**

- 그리고, 이러한 알고리즘에 대한 코드는 다음과 같습니다.

In [ ]:
# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 최대 입력 길이 설정
MAX_LENGTH = 512

# 감정 분석 수행 함수 정의
def get_sentiment_score(text, prev_score=None):
    if len(text) > MAX_LENGTH:
        text = text[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(text)

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    )

    return score

# 긍정적 및 부정적 단어 리스트 정의
positive_words = [
    "happy", "joyful", "excellent", "good", "wonderful", "amazing", "great", "positive", "fortunate", "pleased",
    "delightful", "content", "cheerful", "satisfied", "thrilled", "ecstatic", "merry", "elated", "jubilant", "gleeful"
]

negative_words = [
    "sad", "terrible", "bad", "awful", "horrible", "negative", "unfortunate", "displeased", "angry", "fearful",
    "miserable", "depressed", "unhappy", "downcast", "gloomy", "dismal", "wretched", "heartbroken", "melancholy", "despair"
]

# 중요 문단을 인식하기 위한 키워드 리스트
important_keywords = [
    "conflict", "resolution", "climax", "turning point", "significant", "important", "crucial", "pivotal"
]

# 감정 분석 수행 함수 정의 (중요 문단 인식)
def get_sentiment_score(paragraph, prev_score=None):
    if len(paragraph) > MAX_LENGTH:
        paragraph = paragraph[:MAX_LENGTH]  # 입력 문단을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(paragraph)

    # 문맥에 따라 긍정 또는 부정 가중치 적용
    context_weight = 1.0
    if any(word in paragraph for word in positive_words):
        context_weight = 1.2  # 긍정 문맥 가중치
    elif any(word in paragraph for word in negative_words):
        context_weight = 0.8  # 부정 문맥 가중치

    # 중요 문단 가중치 적용
    important_weight = 1.0
    if any(keyword in paragraph for keyword in important_keywords):
        important_weight = 1.5

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합 및 가중치 적용
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    ) * context_weight * important_weight

    return score

# 첫 번째 문단은 이전 문단이 없으므로 먼저 처리
sentiment_analysis = [get_sentiment_score(paragraphs[0])]

# 나머지 문단 처리
for i in range(1, total_paragraphs):
    prev_score = sentiment_analysis[-1]
    sentiment_analysis.append(get_sentiment_score(paragraphs[i], prev_score))



---



### **4. specific_size와 target_standard_num 계산**  

#### **(1) specific_size와 target_standard_num의 정의**



- 두 변수(specific_size, standard_num)에 대한 정의는 다음과 같습니다.



```
specific_size: 문단을 그룹화하여 그룹 내에 들어가는 문장의 개수입니다.  
단일 문장에 대한 변화량을 시각화하면 변동이 너무 크기 때문에, 이 값을 이용하여 그룹 내의 문장의 감정 극성 평균을 계산할 때 사용됩니다.

```



```
target_standard_num: 목표로 하는 그룹의 수입니다.  
전체 문장의 개수를 target_standard_num에 가장 가깝게 나눌 수 있는 specific_size를 계산합니다.
```

```
따라서 specific_size와 target_standard_num은 데이터를 시각화하기 위해 반드시 필요하나,  
specific_size가 너무 커져버리면 감정 극성 데이터가 왜곡(distort)될 수 있기 때문에 적당히 조절하는 것이 중요합니다.
```



그리고 이를 다음과 같이 계산합니다.  

**[specific_size와 standard_num 계산]**
- **specific_size 계산**: 소설의 총 문단 수를 기반으로 specific_size를 계산합니다.
- **standard_num 계산**: target_standard_num에 가장 근접한 specific_size를 찾습니다.

#### **(2) specific_size와 target_standard_num 값의 조정**

- 하나의 그룹 내에 들어가는 문단의 수가 매우 커지면, 감정 극성이 왜곡될 수 있으므로 이를 방지하기 위한 조정이 필요합니다.  
- 따라서 이 과정에서 specific_size가 20에 가까운 정수가 되도록 하여 분석의 정확성과 시각화의 안정성을 유지합니다. (참고 : 20이라는 숫자는 여러 실험 결과 도출된 결론입니다.)  
- 이를 위해 target_standard_num을 비례식으로 조정하여 specific_size가 적절한 크기로 유지되도록 합니다.  

In [ ]:
# specific_size와 standard_num 계산
def calculate_specific_size(total_paragraphs, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None

    for specific_size in range(1, total_paragraphs + 1):
        standard_num = total_paragraphs // specific_size
        difference = abs(standard_num - target_standard_num)

        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num

        if closest_difference == 0:
            break

    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_paragraphs)



---



### **5. 다양한 평균 계산**  

- 단순 누적 평균, 가중 누적 평균 (EWMA), 이동 평균을 계산합니다. 각 평균의 특징은 다음과 같습니다.

| 평균 종류           | 설명                                                                                 |
|---------------------|--------------------------------------------------------------------------------------|
| 단순 누적 평균      | 모든 문장의 감정 점수를 누적하여 평균을 계산합니다. 이는 전체적인 감정 흐름을 반영합니다.        |
| 가중 누적 평균 (EWMA)| 최근 문장에 더 큰 가중치를 두어 평균을 계산합니다. 이는 최근 감정 변화에 더 민감하게 반응합니다. |
| 이동 평균           | 일정한 크기의 구간으로 나누어 각 구간의 평균을 계산합니다. 이는 구간별 감정 변화를 반영합니다.     |


**[이들의 특징과 차이를 이용한 다른 분야에서의 분석 예시]**  

**- 단순 누적 평균:** 주식 시장의 장기적인 추세 분석에 사용됩니다.  
**- 가중 누적 평균 (EWMA):** 금융에서 변동성 예측, 품질 관리에서 최근의 변화 감지 등에 사용됩니다.  
**- 이동 평균:** 날씨 패턴 분석, 매출 데이터의 계절적 변동 분석 등에 사용됩니다.



- 인문학에서는 이러한 평균의 특징과 차이에 대한 소설 분석 사례는 아직 없습니다.  
따라서, 이번 코드에서 제가 이러한 평균의 특징과 차이를 활용하여 소설의 감정을 더욱 심화적으로 분석해보도록 하겠습니다.

In [ ]:
# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_sentences) + 1)

# 가중 누적 평균 계산 (EWMA)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 이동 평균 계산
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일, 그래프를 시각화할 때 필요함
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# 이동 평균의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg) # 이동 평균의 최대, 최소를 계산하여 감정 극성의 범위를 결정한다.



---



### **6. 가우시안 스무딩 적용**  

#### **(1) 가우시안 스무딩 최종 적용**



- 계산된 평균 값들에 '가우시안 스무딩'을 적용하여 그래프를 부드럽게 만듭니다.  

- 코드를 작성하면서 그래프를 부드럽게 만드는 데 '스플라인 보간법'을 사용했으나, 이는 과적합의 문제가 있었습니다. 따라서 이 문제를 해결하기 위해 중간에 '가우시안 스무딩' 보간법을 적용하였습니다.  

- 아래는 스플라인 보간법(기존에 채택한 보간법)과 가우시안 스무딩(최종 선택된 보간법)의 정의를 설명합니다.


```
* 스플라인 보간법 (Spline Interpolation):

- 정의: 스플라인 보간법은 주어진 데이터 포인트를 부드럽게 연결하는 기법입니다.   
- 적용: 각 구간을 3차 다항식으로 표현하여 연결합니다.
```



```
* 가우시안 스무딩 (Gaussian Smoothing):

- 정의: 가우시안 스무딩은 데이터의 노이즈를 줄이고 부드러운 곡선을 얻기 위해 사용하는 기법입니다.   
이는 특정 패턴을 부드럽게 만드는 데 사용됩니다.
- 적용: 각 데이터 포인트를 중심으로 주변 데이터 포인트들을 고려하여 가중 평균을 계산합니다.
```

- 조금 더 자세히 설명드리겠습니다.  
스플라인 보간법은 노이즈를 줄이면서 주요 패턴을 잘 반영하지 못하는 문제가 있었기 때문에 가우시안 스무딩으로 변경하였습니다.  

- 과적합의 문제란 학습 데이터에 너무 정확히 맞추어 모델이 일반화되지 못하는 현상을 말합니다. 이를 해결하기 위해 가우시안 스무딩을 적용하여 노이즈를 줄이고 주요 패턴을 더 잘 반영합니다.

#### **(2) sigma 값의 동적 설정**

- **sigma 값의 동적 설정**: 문장의 수에 따라 sigma 값을 동적으로 설정합니다.  
이는 텍스트의 길이에 따라 스무딩 강도를 조절하여, 다양한 텍스트 길이에서도 일관된 분석이 가능하도록 합니다.
  - **특징**:
    - **문장의 수에 따른 조정**: 문장의 수가 많을수록 sigma 값을 줄여서 감정 변화를 더 민감하게 반영합니다.
    - **일관된 분석**: 텍스트의 길이에 관계없이 일관된 스무딩 효과를 제공합니다.


In [ ]:
# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_sentences)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)



---



### **7. 감정 변동성 분석**  





- 최신의 문장에 대한 감정을 가장 잘 반영하는 '이동 평균'의 도함수를 구하여 감정 변동성(Sentimental Volatility)을 분석합니다.

- 도함수(감정 변동성 그래프)는 **감정 변화의 속도와 방향**을 의미합니다.  

- 곡선의 길이(arc length)를 계산하여 **감정 변동성의 빈도와 강도를 수치화**합니다. 곡선의 길이가 길수록 **감정 변동의 빈도와 그 강도가 크다(소설 내에서 감정의 변화가 더 자주 발생하고 그 변화의 강도가 더 크다)는 것을 의미**합니다.  

- 그리고, 이러한 arc length를 곡선 길이를 텍스트의 길이에 따라 정규화한 값인 length score로 바꿉니다. 이 값은 감정 변동성의 빈도와 강도를 정량적으로 나타내며, 텍스트의 길이가 달라져도 일관된 기준으로 비교할 수 있습니다.



```
** 정리 **
- 이동 평균의 도함수를 사용하면 감정 변화의 속도와 방향을 파악할 수 있습니다.  
이를 통해서도 소설의 특정 구간에서 감정의 급격한 변화를 식별할 수 있습니다.
- 감정 변동성(Sentimental Volatility) 곡선의 길이는 '감정 변화의 빈도와 강도(얼마나 자주, 얼마나 크게 일어나는지)'를 수치화한 값입니다.
```





```
length score 정의 및 의미
- **정의**: 곡선 길이를 텍스트의 길이에 따라 정규화한 값.
- **의미**: 감정 변동성의 빈도와 강도를 정량적으로 나타냅니다. 이 값이 클수록 감정 변화가 빈번하고 강도가 크다는 것을 의미합니다.
- **장점**: 텍스트의 길이가 달라져도 일관된 기준으로 감정 변동성을 비교할 수 있습니다.
```



In [ ]:
# 감정 변동성 분석 (이동 평균의 도함수 사용)
volatility = np.diff(sentiment_avg_smooth)

# 가우시안 스무딩 적용 (곡률을 더 크게)
volatility_smooth = gaussian_filter1d(volatility, sigma=2)

# 감정 변동성 시각화 및 곡선 길이 계산
plt.figure(figsize=(12, 6))
plt.plot(volatility_smooth, color='#B4C99D', linestyle='-')
plt.axhline(0, color='grey', linestyle='--')  # y=0의 값에 회색 점선 추가
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')

# 곡선 길이 계산 및 정규화된 arc length 계산
compressed_x_volatility = np.linspace(0, total_sentences, num=100)
compressed_volatility_smooth = np.interp(compressed_x_volatility, np.linspace(0, total_sentences, num=len(volatility_smooth)), volatility_smooth)
arc_length = np.sum(np.sqrt(np.diff(compressed_x_volatility)**2 + np.diff(compressed_volatility_smooth)**2))

# 전체 길이 계산
total_length = np.linalg.norm([compressed_x_volatility[-1] - compressed_x_volatility[0], compressed_volatility_smooth[-1] - compressed_volatility_smooth[0]])

# 소설 길이 기반 정규화
novel_length = total_sentences  # 소설의 전체 문장 수
normalized_arc_length = arc_length / novel_length

# 상대적 곡선 길이 계산
relative_arc_length = normalized_arc_length / total_length

# 스케일링 적용
scaled_relative_arc_length = relative_arc_length * 100000

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {scaled_relative_arc_length:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()



---



### **8. Significant Fluctuation Point 식별 및 인접한 구간 합치기**

####**(1) Significant Fluctuation Point (S.F.P.)란?**  


- Significant Fluctuation Point는 소설에서 감정의 변화가 유의미하게 큰 구간입니다.  

- 이 때, Significant Fluctuation Point(S.F.P.)는 **Significant Different Point(S.D.P.)**와 **Significant Volatility Point(S.V.P.)**로 구성됩니다.

####**(2) Significant Different Point (S.D.P.)**

- 가중 누적 평균과 이동 평균의 차이를 계산합니다.  

- 가중 누적 평균은 감정의 변화를 더 민감하게 반영하며, 이동 평균은 전반적인 추세를 반영합니다.  
- 따라서 **두 그래프의 차이가 크면 그 구간에서 감정의 급격한 변화를 의미**합니다.

- 따라서 두 그래프 사이의 차이의 임계값을 설정하여, **임계값 이상의 차이가 발생한 문장의 지점을 탐지**합니다.

In [ ]:
# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.2 * std_dev_difference  # 이동 표준 편차의 1.2배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

####**(3) Significant Volatility Point (S.V.P.)**

- Significant Volatility의 그래프는 이동 평균의 도함수입니다.  

- 따라서, 도함수의 **극점, 함숫값이 급격히 변하는 지점**은 감정의 변화가 큰 지점을 의미하므로, 이 지점을 찾아서 식별합니다.  

In [ ]:
# Sentiment Volatility에서 차이가 크게 나타나는 구간 식별
volatility_diff_indices = []
for i in range(1, len(volatility_smooth) - 1):
    if (volatility_smooth[i] > volatility_smooth[i-1] and volatility_smooth[i] > volatility_smooth[i+1]) or \
       (volatility_smooth[i] < volatility_smooth[i-1] and volatility_smooth[i] < volatility_smooth[i+1]):
        volatility_diff_indices.append(i)

#### **(4) 오차 범위 설정**

- 그리고, '오차 범위'를 설정합니다.  

- 이는 임계값 이상의 차이가 발생한 지점을 기준으로 하여, **그 문단의 전후로 specific_size만큼의 범위를 설정**하여 차이가 크게 나타나는 **'구간'을 만들게 하기 위함**입니다.

In [ ]:
# 오차 범위를 고려한 실제 문단 인덱스 계산
actual_volatility_indices = []
for idx in volatility_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_paragraphs, idx * specific_size + specific_size // 2)
    actual_volatility_indices.append((start_idx, end_idx))

#### **(5) 인접한 구간 합치기**

- 인접한 구간을 합치는 이유는 감정의 변화가 지속적으로 나타나는 구간을 하나의 연속적인 변동으로 보기 위함입니다.

In [ ]:
# 인접한 구간 합치기
merged_volatility_indices = []
for start_idx, end_idx in actual_volatility_indices:
    if not merged_volatility_indices:
        merged_volatility_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_volatility_indices[-1]
        if start_idx <= last_end:
            merged_volatility_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_volatility_indices.append([start_idx, end_idx])

#### **(6) 완성된 구간 표시**

- 차이가 크게 나타나는 구간의 문단의 인덱스를 표시합니다.  

- 인덱스를 통해 어느 문단 지점에서 감정 변화가 일어났는지를 확인할 수 있습니다.

In [ ]:
# 실제 문장 인덱스 기반 텍스트 분석
fig, ax = plt.subplots(figsize=(12, 2))
ax.text(0.5, 0.5, "<Significant Fluctuation Point>\n\nSignificant Different Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_diff_indices]) + "\n\nSignificant Volatility Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_volatility_indices]), horizontalalignment='center', verticalalignment='center', fontsize=12)
ax.axis('off')
plt.show()



---



### **9. 소설에서의 전체적인 감정 분석 시각화**

#### **(1) 가중 합산을 통한 소설 전반의 분위기 시각화**

- 소설의 각 문단에서 계산된 감정 점수를 바탕으로, 소설의 전체적인 분위기를 점수화하고 시각화합니다. 이를 점수화하는 방법은 **가중 합산**을 이용하는 것이고, 아래에 후술하겠습니다.  

- 이를 통해 소설의 전반적인 감정 흐름을 한눈에 파악하고, 긍정적인 부분과 부정적인 부분의 비율을 명확히 비교할 수 있습니다.  

- 이렇게 하면 독자가 소설의 전체 분위기를 쉽게 이해할 수 있고, 감정적인 변화를 효과적으로 분석할 수 있습니다.

#### **(2) 가중 합산을 이용한 점수화 알고리즘**

**문단별 긍정 및 부정 점수 분리:**
- 감정 분석 결과에서 긍정적인 점수와 부정적인 점수를 각각 분리합니다.<br><br>

**긍정 및 부정 점수 합산:**
- 긍정 점수와 부정 점수의 합계를 구합니다. 부정 점수는 절댓값을 사용하여 계산합니다.<br><br>

**가중치 적용:**
- 긍정 점수와 부정 점수 각각에 해당 문단의 수를 가중치로 적용하여 가중합을 계산합니다.<br><br>

**총합 계산:**
- 긍정 점수의 가중합과 부정 점수의 가중합을 더하여 총합을 계산합니다.<br><br>

**백분율로 환산:**
- 긍정 점수와 부정 점수를 전체 점수 대비 백분율로 환산합니다.
- 이는 전체 점수 중에서 긍정 점수와 부정 점수가 각각 차지하는 비율을 나타냅니다.<br><br>




In [ ]:
# 문단별 긍정 및 부정 점수 분리
positive_scores = [score for score in sentiment_analysis if score > 0]
negative_scores = [score for score in sentiment_analysis if score < 0]

# 긍정 및 부정 점수 합산 (절댓값 사용)
positive_score_sum = sum(positive_scores)
negative_score_sum = abs(sum(negative_scores))

# 가중치 적용 (문단 수)
positive_weighted_sum = positive_score_sum * len(positive_scores)
negative_weighted_sum = negative_score_sum * len(negative_scores)

# 총합 계산
total_weighted_sum = positive_weighted_sum + negative_weighted_sum

# 백분율로 환산
positive_final_score = (positive_weighted_sum / total_weighted_sum) * 100
negative_final_score = (negative_weighted_sum / total_weighted_sum) * 100

# 막대 바 차트 시각화 코드
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_final_score, negative_final_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.2f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_final_score, negative_final_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()



---

